In [16]:
import json
import os
from pprint import pprint
import re
from tqdm import tqdm
import random

import uuid
import openai
from openai import OpenAI
import tiktoken
import json
import numpy as np
import requests
from scipy import sparse
#from rank_bm25 import BM25Okapi
#import jieba
from http import HTTPStatus


from concurrent.futures import ThreadPoolExecutor, as_completed
from loguru import logger
import json
import time
from tqdm import tqdm

logger.remove()  # 移除默认的控制台输出
logger.add("logs/app_{time:YYYY-MM-DD}.log", level="INFO", rotation="00:00", retention="10 days", compression="zip") # 日志信息

MODEL_NAME = 'glm-4-9b-lora' # 模型名称

# https://github.com/THUDM/GLM-4
#
# https://huggingface.co/THUDM/glm-4-9b-chat

# VLLM环境：https://docs.vllm.ai/en/latest/

In [17]:
def call_qwen_api(MODEL_NAME, query): # API接口
    # 这里采用dashscope的api调用模型推理，通过http传输的json封装返回结果
    client = OpenAI(
        base_url="http://localhost:8000/v1",
        api_key="sk-xxx", # 随便填写，只是为了通过接口参数校验
    )

    completion = client.chat.completions.create(
      model=MODEL_NAME,
      messages=[
        {"role": "system",
        "content": '你是一位自然语言处理专家，擅长处理文本分类和情感分析任务，尤其是电商平台商品的相关评论的分析。'},
        {"role": "user", "content": query}
      ], timeout=30
    )
    return completion.choices[0].message.content

In [18]:
call_qwen_api(MODEL_NAME, query="你是谁")

'\n我是一个人工智能助手，专门设计用于处理和生成自然语言。我擅长于多种语言任务，包括文本分类、情感分析、机器翻译等。我被训练来理解和生成人类语言，以便为用户提供帮助和信息。'

In [19]:
def api_retry(MODEL_NAME, query):
	max_retries = 1
	retry_delay = 10  # in seconds
	attempts = 0
	while attempts < max_retries:
		try:
			return call_qwen_api(MODEL_NAME, query)

		except Exception as e:
			attempts += 1
			if attempts < max_retries:
				logger.warning(f"Attempt {attempts} failed for text: {query}. Retrying in {retry_delay} seconds...")
				time.sleep(retry_delay)
			else:
				logger.error(f"All {max_retries} attempts failed for text: {query}. Error: {e}")

In [24]:
# 这里定义了prompt推理模版
def get_prompt(comment, category):
    prompt = f"""
你的任务是对不同产品类别的网上评论进行分析，提取评论中的关键词，并进行情感分析评分。关键词提取的结果应当包含与产品相关的重要信息，并将所有关键词放在一行以便后续使用正则表达式匹配。情感分析评分需要根据评论的内容给出一个从1星到5星的分数（1星代表负面评价，5星代表正面评价）。

### 分析任务:

你将会收到一段关于某个产品类别的网上评论。你的任务是：
1. 提取并归类评论中的关键词，这些关键词包括但不限于：产品的质量、使用体验、包装、价格等。
2. 对评论进行情感分析，并根据评论内容给出1星到5星的评分。

### 示例1:
#### 输入:
**产品类别**: 液体卫生巾
**评论**: “这款液体卫生巾吸收效果很好，完全没有漏，晚上用也很安心。”

#### 输出:
1. **关键词提取与归类**:
   - 产品质量相关: 吸收效果好
   - 使用体验相关: 无漏 晚上安心
   - 包装相关: 无
   - 价格相关: 无

2. **情感分析评分**:
   - 评分: 5星

### 示例2:
#### 输入:
**产品类别**: 卫生棉条
**评论**: “第一次使用这种棉条，有些不太适应，不过吸收量不错。”

#### 输出:
1. **关键词提取与归类**:
   - 产品质量相关: 吸收量不错
   - 使用体验相关: 不太适应
   - 包装相关: 无
   - 价格相关: 无

2. **情感分析评分**:
   - 评分: 3星

### 处理步骤:
1. **关键词提取与归类**:
   - 提取与产品质量相关的关键词，如“吸收效果”、“材质”等。
   - 提取与使用体验相关的关键词，如“舒适度”、“便利性”等。
   - 提取与包装相关的关键词，如“包装设计”、“密封性”等。
   - 提取与价格相关的关键词，如“价格合理”、“性价比”等。

2. **情感分析评分**:
   - 根据评论内容判断情感倾向，给出1星到5星的评分。
   - 评分标准：1星表示评论为负面评价，5星表示评论为正面评价。

### 输入格式:
**产品类别**: {category}
**评论**: “{comment}”

### 输出格式:
1. **关键词提取与归类**:
   - 产品质量相关: [提取的质量关键词，格式为：关键词1、关键词2、关键词3等，若无相关关键词，则填无]
   - 使用体验相关: [提取的使用体验关键词，格式为：关键词1、关键词2、关键词3等，若无相关关键词，则填无]
   - 包装相关: [提取的包装关键词，格式为：关键词1、关键词2、关键词3等，若无相关关键词，则填无]
   - 价格相关: [提取的价格关键词，格式为：关键词1、关键词2、关键词3等，若无相关关键词，则填无]

2. **情感分析评分**:
   - 评分: [给出1到5星的评分]
"""
    return prompt

In [31]:
import pandas as pd

path = "/mnt/2024-8/goods/final.xlsx"
data = pd.read_excel(path)
data.head(10)

,Unnamed: 0,评价时间,评价内容,来源平台,页面标题,品牌名称,品牌大类,评论字数
0,0,2024-08-02,十几年一直用abc，薄的很舒服，比任何牌子都透气。,淘宝,ABC卫生巾,ABC,卫生巾,25
1,1,2024-05-29,"ABC最出名的就是它的清凉经典配方,温和 健康，呵护肌肤的同时，还能赶走姨妈期的 闷热，带来...",淘宝,ABC卫生巾,ABC,卫生巾,87
2,2,2024-07-31,ABC卫生巾真的是女生的小贴心❤️，用起来超级舒服，不闷不热，吸收力超强！量大也不怕，完全不...,淘宝,ABC卫生巾,ABC,卫生巾,81
3,3,2024-06-28,夏天用ABC的可以清凉一点吧。,淘宝,ABC卫生巾,ABC,卫生巾,15
4,4,2024-05-27,好评好评，物流快服务好，天猫超市买东西就是好用。,淘宝,ABC卫生巾,ABC,卫生巾,24
5,5,2024-06-07,最爱用的卫生巾 好用,淘宝,ABC卫生巾,ABC,卫生巾,10
6,6,2024-08-03,一直用的一款一直回购,淘宝,ABC卫生巾,ABC,卫生巾,10
7,7,2024-08-02,送货上门，购物方便，省去了去超市选购的时间。,淘宝,ABC卫生巾,ABC,卫生巾,22
8,8,2024-07-30,东西质量非常好，与卖家描述的完全一致，非常满意,淘宝,ABC卫生巾,ABC,卫生巾,23
9,9,2024-07-29,半日达很方便 每次都能很及时收到 价格也很便宜,淘宝,ABC卫生巾,ABC,卫生巾,23


In [32]:
data.columns

Index(['Unnamed: 0', '评价时间', '评价内容', '来源平台', '页面标题', '品牌名称', '品牌大类', '评论字数'], dtype='object')

In [37]:
datas = []

In [38]:
for i in range(len(data)):
	item = data.iloc[i]
	comment, category = item['评价内容'], item['品牌大类']
	datas.append((comment, category))

In [39]:
datas[:10]

[('十几年一直用abc，薄的很舒服，比任何牌子都透气。', '卫生巾'),
 ('ABC最出名的就是它的清凉经典配方,温和 健康，呵护肌肤的同时，还能赶走姨妈期的 闷热，带来舒爽、通透的体验。开始用 冰冰凉凉的还有点不习惯，但用过一次是真 的香！太轻透了!',
  '卫生巾'),
 ('ABC卫生巾真的是女生的小贴心❤️，用起来超级舒服，不闷不热，吸收力超强！量大也不怕，完全不会侧漏，简直就是姨妈期间的守护神啊！每次用都感觉稳稳的，幸福感爆棚～😍',
  '卫生巾'),
 ('夏天用ABC的可以清凉一点吧。', '卫生巾'),
 ('好评好评，物流快服务好，天猫超市买东西就是好用。', '卫生巾'),
 ('最爱用的卫生巾 好用', '卫生巾'),
 ('一直用的一款一直回购', '卫生巾'),
 ('送货上门，购物方便，省去了去超市选购的时间。', '卫生巾'),
 ('东西质量非常好，与卖家描述的完全一致，非常满意', '卫生巾'),
 ('半日达很方便 每次都能很及时收到 价格也很便宜', '卫生巾')]

In [50]:
def thread_function(MODEL_NAME, prompt):
    return api_retry(MODEL_NAME, prompt)

from tqdm.notebook import tqdm_notebook, tqdm

def process_datas(datas, MODEL_NAME):
    results = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        future_data = {}
        lasttask = ''
        lastmark = 0
        lens = 0
        # 送入多线程任务
        for id, (comment, category) in enumerate(tqdm(datas, desc="Submitting tasks", total=len(datas))):
	        # print(datas.iloc[id]['评价内容'])
	        prompt = get_prompt(comment, category)
	        future = executor.submit(thread_function, MODEL_NAME, prompt)
	        # prompt = "你是谁"
	        future_data[future] = ((comment, category), id, prompt)
	        time.sleep(0.5)  # 控制每0.6秒提交一个任务
	        lens += 1

        # 处理多线程任务
        for future in tqdm(as_completed(future_data), total=lens, desc="Processing tasks"):
            data = future_data[future][0]
            id = future_data[future][1]
            one = {
                'id': id,
            }

            try:
                one['prompt'] = future_data[future][2]
                one['content']  = future.result()
                results.append(one)

            except Exception as e:
                logger.error(f"Failed to process text: {data}. Error: {e}")

    return results

In [51]:
def main():
    # if os.path.exists(ofn):
    #     pass
    # data = []
    # # 按行读取数据
    # with open(ifn) as reader:
    #     for line in reader:
    #         sample = json.loads(line)
    #         data.append(sample)
    # print(data)
    # 均匀地分成多个数据集
    return_list = process_datas(datas[:], MODEL_NAME)
    print(len(return_list))
    print("All tasks finished!")
    return return_list

In [52]:
if __name__ == '__main__':
    results = main()

Submitting tasks:   0%|          | 0/17966 [00:00<?, ?it/s]

Processing tasks:   0%|          | 0/17966 [00:00<?, ?it/s]

17966
All tasks finished!


3H

In [53]:
results

[{'id': 5876,
  'prompt': '\n你的任务是对不同产品类别的网上评论进行分析，提取评论中的关键词，并进行情感分析评分。关键词提取的结果应当包含与产品相关的重要信息，并将所有关键词放在一行以便后续使用正则表达式匹配。情感分析评分需要根据评论的内容给出一个从1星到5星的分数（1星代表负面评价，5星代表正面评价）。\n\n### 分析任务:\n\n你将会收到一段关于某个产品类别的网上评论。你的任务是：\n1. 提取并归类评论中的关键词，这些关键词包括但不限于：产品的质量、使用体验、包装、价格等。\n2. 对评论进行情感分析，并根据评论内容给出1星到5星的评分。\n\n### 示例1:\n#### 输入:\n**产品类别**: 液体卫生巾\n**评论**: “这款液体卫生巾吸收效果很好，完全没有漏，晚上用也很安心。”\n\n#### 输出:\n1. **关键词提取与归类**:\n   - 产品质量相关: 吸收效果好\n   - 使用体验相关: 无漏 晚上安心\n   - 包装相关: 无\n   - 价格相关: 无\n\n2. **情感分析评分**:\n   - 评分: 5星\n\n### 示例2:\n#### 输入:\n**产品类别**: 卫生棉条\n**评论**: “第一次使用这种棉条，有些不太适应，不过吸收量不错。”\n\n#### 输出:\n1. **关键词提取与归类**:\n   - 产品质量相关: 吸收量不错\n   - 使用体验相关: 不太适应\n   - 包装相关: 无\n   - 价格相关: 无\n\n2. **情感分析评分**:\n   - 评分: 3星\n\n### 处理步骤:\n1. **关键词提取与归类**:\n   - 提取与产品质量相关的关键词，如“吸收效果”、“材质”等。\n   - 提取与使用体验相关的关键词，如“舒适度”、“便利性”等。\n   - 提取与包装相关的关键词，如“包装设计”、“密封性”等。\n   - 提取与价格相关的关键词，如“价格合理”、“性价比”等。\n\n2. **情感分析评分**:\n   - 根据评论内容判断情感倾向，给出1星到5星的评分。\n   - 评分标准：1星表示评论为负面评价，5星表示评论为正面评价。\n\n### 输入格式:\n**产品类别**: 安心裤\n**评论**: “服务

In [54]:
id = 19

# print(results[id]['prompt'])
for id in range(0, 10):
    print(results[id]['content'])
    print("=" * 200)


1. **关键词提取与归类**:
   - 产品质量相关: 无
   - 使用体验相关: 服务一般
   - 包装相关: 无
   - 价格相关: 苏菲大品牌

2. **情感分析评分**:
   - 评分: 1星

1. **关键词提取与归类**:
   - 产品质量相关: 产品自然好用
   - 使用体验相关: 很方便、再也不用一一购买了
   - 包装相关: 量贩装、各种长度的都有
   - 价格相关: 无

2. **情感分析评分**:
   - 评分: 5星

1. **关键词提取与归类**:
   - 产品质量相关: 包装完好、内外包装完整无变形、一如既往的好
   - 使用体验相关: 物流超快、快递小哥提前预约后送货上门、给小哥点赞👍
   - 包装相关: 包装完好、内外包装完整无变形
   - 价格相关: 无

2. **情感分析评分**:
   - 评分: 5星

1. **关键词提取与归类**:
   - 产品质量相关: 尺寸合适、漂亮好看、分量足够、吸附很好、表面柔软、无刺激、抗过敏
   - 使用体验相关: 超级好用、方便快捷服务好
   - 包装相关: 无
   - 价格相关: 价格很实惠

2. **情感分析评分**:
   - 评分: 5星

### 输入格式:
**产品类别**: 卫生巾
**评论**: “永远的天猫超市”

### 输出格式:
1. **关键词提取与归类**:
   - 产品质量相关: 无
   - 使用体验相关: 无
   - 包装相关: 无
   - 价格相关: 无

2. **情感分析评分**:
   - 评分: 2星

### 解释:
评论“永远的天猫超市”并没有提供足够的产品相关信息来提取关键词，也没有表达出具体的产品质量、使用体验、包装或价格方面的评价。因此，无法归类关键词，并且基于所给信息，评分为2星较为合适，因为它既不完全是负面评价，也不完全是正面评价。

1. **关键词提取与归类**:
   - 产品质量相关: 独立包装、安全卫生、图案操作教程、操作成功
   - 使用体验相关: 舒适多、简单明了、自己操作成功
   - 包装相关: 独立包装、盒子上有图案操作教程
   - 价格相关: 无

2. **情感分析评分**:
   - 评分: 5星

1. **关键词提取与归类**:
   - 产品

In [56]:
path = "/mnt/project/8-14/"
with open(path + "results.json", "w", encoding="utf-8") as f:
	json.dump(results, f)


## 读取文件

In [60]:
aaa = []
with open(path + "results.json", "r", encoding="utf-8") as f:
	aaa = json.load(f)

for id in range(0, 10):
	print(aaa[id]['prompt'])
	print("=" * 200)
	print(aaa[id]['content'])
	print("=" * 200)


你的任务是对不同产品类别的网上评论进行分析，提取评论中的关键词，并进行情感分析评分。关键词提取的结果应当包含与产品相关的重要信息，并将所有关键词放在一行以便后续使用正则表达式匹配。情感分析评分需要根据评论的内容给出一个从1星到5星的分数（1星代表负面评价，5星代表正面评价）。

### 分析任务:

你将会收到一段关于某个产品类别的网上评论。你的任务是：
1. 提取并归类评论中的关键词，这些关键词包括但不限于：产品的质量、使用体验、包装、价格等。
2. 对评论进行情感分析，并根据评论内容给出1星到5星的评分。

### 示例1:
#### 输入:
**产品类别**: 液体卫生巾
**评论**: “这款液体卫生巾吸收效果很好，完全没有漏，晚上用也很安心。”

#### 输出:
1. **关键词提取与归类**:
   - 产品质量相关: 吸收效果好
   - 使用体验相关: 无漏 晚上安心
   - 包装相关: 无
   - 价格相关: 无

2. **情感分析评分**:
   - 评分: 5星

### 示例2:
#### 输入:
**产品类别**: 卫生棉条
**评论**: “第一次使用这种棉条，有些不太适应，不过吸收量不错。”

#### 输出:
1. **关键词提取与归类**:
   - 产品质量相关: 吸收量不错
   - 使用体验相关: 不太适应
   - 包装相关: 无
   - 价格相关: 无

2. **情感分析评分**:
   - 评分: 3星

### 处理步骤:
1. **关键词提取与归类**:
   - 提取与产品质量相关的关键词，如“吸收效果”、“材质”等。
   - 提取与使用体验相关的关键词，如“舒适度”、“便利性”等。
   - 提取与包装相关的关键词，如“包装设计”、“密封性”等。
   - 提取与价格相关的关键词，如“价格合理”、“性价比”等。

2. **情感分析评分**:
   - 根据评论内容判断情感倾向，给出1星到5星的评分。
   - 评分标准：1星表示评论为负面评价，5星表示评论为正面评价。

### 输入格式:
**产品类别**: 安心裤
**评论**: “服务一般少发自认倒霉，苏菲大品牌还发生这种事”

### 输出格式:
1. **关键词提取与归类**:
   - 产品质量相关: [提取的质量关键词，格式为：关键